In [33]:
import numpy as np
import pandas as pd

## Activation func
def relu(x):
    return np.maximum(x, 0)

def relu_derivative(x):
    return (x > 0).astype(float)

In [ ]:
## Data Reading and Cleaning

encoder = {"NEAR BAY": 1, "<1H OCEAN": 2, "INLAND": 3, "NEAR OCEAN": 4}

dataset = pd.read_csv("housing.csv")

dataset["total_bedrooms"] = dataset["total_bedrooms"].fillna(dataset["total_bedrooms"].mean())
dataset["ocean_proximity"] = dataset["ocean_proximity"].map(encoder).fillna(0)
y = dataset["median_house_value"].values.reshape(-1, 1)
dataset.drop(labels="median_house_value", axis=1, inplace=True)

In [53]:
## Train/Test Split

threshold = int(0.7 * len(y))

X_train = dataset[:threshold]
X_test = dataset[threshold:]

y_train = y[:threshold]
y_test = y[threshold:]

In [ ]:
## Normalizing
X_mean = np.mean(X_train, axis=0)
X_std = np.std(X_train, axis=0)
X_train_norm = (X_train - X_mean) / X_std

y_mean = np.mean(y_train, axis=0)
y_std = np.std(y_train, axis=0)
y_train_norm = (y_train - y_mean) / y_std

In [88]:
## Training
n = len(y_train)
epochs = 20000
learning_rate = 0.03

## Weights
w1 = np.random.randn(9, 4) * 0.01
b1 = np.zeros((1, 4)) * 0.01

w2 = np.random.randn(4, 4) * 0.01
b2 = np.zeros((1, 4)) * 0.01

w3 = np.random.randn(4, 1) * 0.01
b3 = np.zeros((1, 1)) * 0.01

for i in range(epochs):
    ## Forward Prop
    h1 = relu(np.dot(X_train_norm, w1) + b1)
    h2 = relu(np.dot(h1, w2) + b2)
    y_train_hat_norm = np.dot(h2, w3) + b3

    ## Back Propogation
    loss = np.mean((y_train_norm - y_train_hat_norm)**2)

    dl_dy_hat = (2/n) * (y_train_hat_norm - y_train_norm)
    dl_w3 = np.dot(h2.T, dl_dy_hat)
    dl_b3 = np.sum(dl_dy_hat, axis=0, keepdims=True)

    dl_h2 = np.dot(dl_dy_hat, w3.T) * relu_derivative(h2)
    dl_w2 = np.dot(h1.T, dl_h2)
    dl_b2 = np.sum(dl_h2, axis=0, keepdims=True)

    dl_h1 = np.dot(dl_h2, w2.T) * relu_derivative(h1)
    dl_w1 = np.dot(X_train_norm.T, dl_h1)
    dl_b1 = np.sum(dl_h1, axis=0, keepdims=True)

    ## Subtracting weights
    w3 -= learning_rate * dl_w3
    b3 -= learning_rate * dl_b3

    w2 -= learning_rate * dl_w2
    b2 -= learning_rate * dl_b2

    w1 -= learning_rate * dl_w1
    b1 -= learning_rate * dl_b1

    if (i % 100 == 0):
        print(f"Epoch: {i}, Loss: {loss}")

Epoch: 0, Loss: 0.9999906468191553
Epoch: 100, Loss: 0.9999848405760453
Epoch: 200, Loss: 0.9999794932920532
Epoch: 300, Loss: 0.9999717462420796
Epoch: 400, Loss: 0.9999600891571233
Epoch: 500, Loss: 0.9999416614651607
Epoch: 600, Loss: 0.9999106522400087
Epoch: 700, Loss: 0.9998541253830129
Epoch: 800, Loss: 0.9997394711612647
Epoch: 900, Loss: 0.9994677815508167
Epoch: 1000, Loss: 0.9986461508986999
Epoch: 1100, Loss: 0.9947703937286227
Epoch: 1200, Loss: 0.9415471169610895
Epoch: 1300, Loss: 0.4541867672229473
Epoch: 1400, Loss: 0.39084743876097905
Epoch: 1500, Loss: 0.3595651602106662
Epoch: 1600, Loss: 0.3432273816141177
Epoch: 1700, Loss: 0.3308757532348527
Epoch: 1800, Loss: 0.3200980386751607
Epoch: 1900, Loss: 0.31054160946572296
Epoch: 2000, Loss: 0.30277890180071926
Epoch: 2100, Loss: 0.29682702780708387
Epoch: 2200, Loss: 0.29239515249955245
Epoch: 2300, Loss: 0.28937242541673924
Epoch: 2400, Loss: 0.2872765065058678
Epoch: 2500, Loss: 0.2857173097558369
Epoch: 2600, Loss:

In [ ]:
## Testing

X_test_norm = (X_test - X_mean) / X_std

h1 = relu(np.dot(X_test_norm, w1) + b1)
h2 = relu(np.dot(h1, w2) + b2)
y_test_hat_norm = np.dot(h2, w3) + b3

y_test_hat = (y_test_hat_norm * y_std) + y_mean
loss = np.abs(y_test - y_test_hat)

for i in range(100):
    print(f'Predicted: {y_test_hat[i][0]:.2f}, Actual: {y_test[i][0]}, Difference: {loss[i][0]:.2f}')

Predicted: 408282.31, Actual: 500001.0, Difference: 91718.69
Predicted: 271564.30, Actual: 450000.0, Difference: 178435.70
Predicted: 330649.86, Actual: 332400.0, Difference: 1750.14
Predicted: 380103.86, Actual: 394400.0, Difference: 14296.14
Predicted: 276303.06, Actual: 311900.0, Difference: 35596.94
Predicted: 234743.91, Actual: 376000.0, Difference: 141256.09
Predicted: 232225.96, Actual: 360300.0, Difference: 128074.04
Predicted: 298652.87, Actual: 410000.0, Difference: 111347.13
Predicted: 323085.40, Actual: 500000.0, Difference: 176914.60
Predicted: 310744.29, Actual: 500001.0, Difference: 189256.71
Predicted: 301304.65, Actual: 500001.0, Difference: 198696.35
Predicted: 297095.36, Actual: 405200.0, Difference: 108104.64
Predicted: 486128.15, Actual: 500001.0, Difference: 13872.85
Predicted: 205344.43, Actual: 488900.0, Difference: 283555.57
Predicted: 272951.91, Actual: 381300.0, Difference: 108348.09
Predicted: 270517.89, Actual: 475000.0, Difference: 204482.11
Predicted: 284